In [1]:
import PySimpleGUI as sg
import lightkurve as lk
import numpy as np
import pandas as pd

In [ ]:
####### THEME #######
sg.theme('DarkGray1')
#####################

#LAYOUT
layout = [
    [sg.Radio('Target', 'tgt' , key='-TARGET_RADIO-', default=True), sg.Input('', key='-TARGET_INPUT-', size=(45, 0), pad=(51, 5))],
    [sg.Radio('List of Targets', 'tgt', key='-TARGET_LIST_RADIO-'), sg.Input(key='-TARGET_FILE-'), sg.FileBrowse(enable_events=True, size=(12, 0))],
    [sg.HSeparator()],
    [sg.Radio('Sector', 'sct', key='-SEC_RADIO-', default=True), sg.Input('', key='-SEC_INPUT-', size=(10, 0), pad=(51, 5))],
    [sg.Radio('List of Sectors', 'sct', key='-SEC_LIST_RADIO-'), sg.Input('', key='-SEC_LIST_INPUT-', pad=(5, 5))],
    [sg.Radio('All Sectors (merged)', 'sct', key='-ALL_SEC_RADIO-')],
    [sg.HSeparator()],
    [sg.Push(), sg.Button('DOWNLOAD', key='-DOWNLOAD_BUTTON-', size=(12, 2)), sg.Push()],
    [sg.ProgressBar(100, size=(500, 15), bar_color=('green', 'lightgrey'))]
    
]

# FUNCTIONALITIES
def single_target(tic, sector):
    search = lk.search_lightcurve(target=f'TIC {tic}', sector=sector, author='SPOC', exptime=120)
    lc = search.download().normalize().remove_outliers(sigma=3.5)
    time = lc.time.value
    flux = lc.flux.value
    df = pd.DataFrame(np.column_stack([time,flux]), columns=['Time', 'Flux'] )
    return df
    


# JANELA
window = sg.Window('TL Download Project', layout=layout, size=(550, 300))


while True:
    event, value = window.read()
    
    if event == sg.WIN_CLOSED:
        break
        
    if event == '-DOWNLOAD_BUTTON-':
        ### Download the specified sector of the specified target ###
        if value['-TARGET_RADIO-'] == True and value['-SEC_RADIO-'] == True:
            tic = value['-TARGET_INPUT-']
            sector = value['-SEC_INPUT-']
            data = single_target(tic, sector)
            data.to_csv(f'{tic.zfill(10)}_{str(sector).zfill(2)}.dat', sep=' ', index=None, header=None)
            
            sg.popup('File saved successfully!')
            
        
window.close()

# Versão com janelas múltiplas

In [ ]:
### FUNCIONALIDADES ###
def search(tic):
    sector_list = []
    search = lk.search_lightcurve(target=f'TIC {tic}', author='SPOC', exptime=120)
    for i in range(len(search)):
        sector_list.append(search.table['mission'][i][12:])
    
    return sector_list


### LAYOUT DAS JANELAS ###
def main_window():
    sg.theme('DarkGray1')
    layout = [
        [sg.Text('Target:'), sg.Input('', key='-TARGET_INPUT-', size=(51, 0)), sg.Button('Search', key='-SEARCH_BUTTON-', size=(12, 0))],
        [sg.HSeparator()],
        [sg.Text('List of Tagets:'), sg.Input(key='-TARGET_FILE-'), sg.FileBrowse(enable_events=True, size=(12, 0))],
        [sg.Radio('All Sectors', 'sec', key='-ALL_SECTORS_RAD-', default=True)],
        [sg.Radio('Choose Sectors:', 'sec', key='-SECTORS_RAD-'), sg.Text('Start:'), sg.Input('', size=(5, 0)), sg.Text('Final:'), sg.Input('', size=(5, 0))],
        [sg.HSeparator()],
        [sg.Push(), sg.Checkbox('All Sectors (MERGED)'), sg.Push()],
        [sg.HSeparator()],
        [sg.Text('')],
        [sg.Push(), sg.Button('DOWNLOAD', key='-DOWNLOAD_BUTTON-', size=(12, 2)), sg.Push()]
    ]
    return sg.Window('TLC Download Project', layout=layout, size=(550, 300), finalize=True)

def sectors_window():
    sg.theme('DarkGray1')
    layout = [
        [sg.Text('Setores Observados:')],
        #[sg.Push(), sg.Button('Cancel', key='-CANCEL_BUTTON-'), sg.Push()]
    ]
    return sg.Window('List of Sectors', layout=layout, size=(250, 100), finalize=True)

### CRIAR JANELAS ###
window_1, window_2 = main_window(), None

while True:
    window, event, value = sg.read_all_windows()
    
    if window == window_1 and event == sg.WIN_CLOSED:
        break
    elif window == window_1 and event == '-SEARCH_BUTTON-':
        tic = value['-TARGET_INPUT-']
        data = search(tic)
        window_2 = sectors_window()
        for i in range(len(data)):
            window_2.extend_layout(window_2, [[sg.Checkbox(''), sg.Text(f'Sec {data[i]}')]])
    elif window == window_2 and event == sg.WIN_CLOSED:
        break
    elif window == window_2 and event == '-CANCEL_BUTTON-':
        window_2.hide() 

        
window.close()

In [ ]:
def search(tic):
    sector_list = []
    search = lk.search_lightcurve(target=f'TIC {tic}', author='SPOC', exptime=120)
    for i in range(len(search)):
        sector_list.append(search.table['mission'][i][12:])
    
    return sector_list

In [ ]:
data = search(137608661)
data